In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import argparse
import random
random.seed(577)

import numpy as np
np.random.seed(577)
import pandas as pd

import torch
torch.set_default_tensor_type(torch.FloatTensor)
torch.use_deterministic_algorithms(True)
torch.manual_seed(577)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.optim as optim

from neural_arch import F1_Loss_Fn, NeuralNetwork, DAN



import gensim.downloader as api


import torch.nn as nn
from torch.utils.data import DataLoader, random_split


from Module.utils import MyDataset, custom_collate

In [2]:
cyber_bulling_df = pd.read_csv('dataset\cyberbullying_tweets.csv')

In [3]:
cyber_bulling_df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [4]:
cyber_bulling_df['cyberbullying_type'] = np.where(cyber_bulling_df['cyberbullying_type'] == "not_cyberbullying", 0, 1)

In [5]:
cyber_bulling_df['cyberbullying_type'].value_counts()

1    39747
0     7945
Name: cyberbullying_type, dtype: int64

In [6]:
39747 / 7945

5.00276903713027

In [7]:
sms_df = pd.read_csv('dataset\sms_dataset.csv')

In [8]:
sms_df['label'].value_counts()

0    4827
1     747
Name: label, dtype: int64

In [9]:
X = sms_df.drop('label', axis=1)
y = sms_df['label']

In [10]:
# call glove-wiki and make vocaburary 
glove_embs = api.load("glove-wiki-gigaword-50")
vocab = {word: index for index, word in enumerate(glove_embs.index_to_key)}
emb_weights = torch.FloatTensor(glove_embs.vectors)  # Convert to PyTorch tensor
embedding_layer = torch.nn.Embedding.from_pretrained(emb_weights, freeze=False)
n_embed, d_embed = emb_weights.size()

In [11]:
#define model
user_input = input("Write nn for neural network, ")

if user_input == "nn":
    model = NeuralNetwork(n_embed= n_embed, d_embed= d_embed, d_hidden=256, embedding_layer= emb_weights)
else:
    model = DAN(n_embed=n_embed, d_embed= d_embed, d_hidden=256, embedding_layer= emb_weights)
    


In [12]:
# loss function and optimizer
loss_fn = F1_Loss_Fn()  # F1_apprx_loss fn
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [14]:
# Load the datasets and set train size and test size
full_dataset = MyDataset(X, y, vocab)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate)

In [15]:
criterion = F1_Loss_Fn()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [17]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import nltk
from nltk.tokenize import word_tokenize

def custom_collate(batch):
    sentence = [item['sentence'] for item in batch]
    label = [item['label'] for item in batch]
    sentence = pad_sequence(sentence, batch_first=True, padding_value=0)
    return {
        'sentence': sentence,
        'label': label
    }

def sentence_to_indices(sentence, vocab):
    # Tokenize the sentence
    tokens = word_tokenize(sentence)
    
    # Convert each word to its index in the vocabulary
    unknown_index = vocab.get('<unk>', -1)  # Use -1 or another suitable default if '<unk>' is not defined
    indices = [vocab.get(token, unknown_index) for token in tokens]
    
    return indices

class MyDataset(Dataset):
    def __init__(self, X, y, vocab):
        self.text = X
        self.labels = y
        self.vocab = vocab
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sentence = sentence_to_indices(self.text.iloc[idx], self.vocab)
        label = self.labels.iloc[idx]
        
        return {
            'sentence': sentence,
            'label': label
        }


In [18]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        sentence = batch['sentence']
        label = batch['label']
        

        

400000


AttributeError: 'Series' object has no attribute 'lower'

In [ ]:
print(sentence)

[sms    Urgent! Please call 09066612661 from your land...
Name: 2730, dtype: object, sms    Gain the rights of a wife.dont demand it.i am ...
Name: 4852, dtype: object, sms    S:)but he had some luck.2 catches put down:)\n
Name: 3699, dtype: object, sms    Ur ringtone service has changed! 25 Free credi...
Name: 268, dtype: object, sms    I wanna watch that movie\n
Name: 1802, dtype: object, sms    Today's Offer! Claim ur £150 worth of discount...
Name: 850, dtype: object, sms    Why do you ask princess?\n
Name: 891, dtype: object, sms    What time you thinkin of goin?\n
Name: 3441, dtype: object, sms    Then ü ask dad to pick ü up lar... Ü wan 2 sta...
Name: 3405, dtype: object, sms    Ok lar... Joking wif u oni...\n
Name: 1, dtype: object, sms    Congrats ! Treat pending.i am not on mail for ...
Name: 2193, dtype: object]
